In [79]:
using DrWatson
@quickactivate "HEMI" 
using HEMI 
using DataFrames, Chain
using CSV, StringEncodings
using Plots

## Métricas de evaluación histórica

In [80]:
# Directorios de resultados 
config_savepath = datadir("results", "mse-combination", "Esc-E-Scramble-OptMAI")
test_savepath = datadir("results", "mse-combination", "Esc-E-Scramble-OptMAI", "testdata")

# CountryStructure con datos hasta período de evaluación 
gtdata_eval = gtdata[Date(2020, 12)]

UniformCountryStructure{2, Float32, Float32} con 2 bases
|─> VarCPIBase{Float32, Float32}: 120 períodos × 218 gastos básicos Jan-2001-Dec-2010
|─> VarCPIBase{Float32, Float32}: 120 períodos × 279 gastos básicos Jan-2011-Dec-2020


In [81]:
# Obtener trayectorias de inflación 
testconfig = wload(joinpath(config_savepath, "cv_test_config_125k.jld2"), "testconfig")
testdata = wload(joinpath(test_savepath, savename(testconfig)))

Dict{String, Any} with 9 entries:
  "dates_20"  => Date("2001-12-01"):Month(1):Date("2020-12-01")
  "gitcommit" => "ad880e839f10ad5561c027dc147f1263b8844bbd_dirty"
  "config"    => CrossEvalConfig{InflationTotalRebaseCPI, ResampleScrambleVarMo…
  "param_20"  => Float32[5.42724, 5.21743, 5.10553, 5.04973, 5.00705, 4.97791, …
  "dates_18"  => Date("2001-12-01"):Month(1):Date("2018-12-01")
  "infl_20"   => Float32[6.16477 5.89241 … 4.50073 6.01093; 5.89446 5.82571 … 4…
  "infl_18"   => Float32[6.16477 5.89241 … 4.50073 5.9893; 5.89446 5.82571 … 4.…
  "gitpatch"  => "diff --git a/scripts/mse-combination/Esc-E-Scramble-OptMAI/ge…
  "param_18"  => Float32[5.42724, 5.21743, 5.10553, 5.04973, 5.00705, 4.97791, …

In [82]:
# Trayectorias de inflación y parámetrica
tray_infl = testdata["infl_20"]
tray_infl_pob = testdata["param_20"];

### Métricas de los 4 períodos

In [83]:
inflfns = [testconfig.inflfn.functions...]
EVAL_PERIODS = [GT_EVAL_B00, GT_EVAL_T0010, GT_EVAL_B10, CompletePeriod()]

indv_results = mapreduce(vcat, 1:length(inflfns)) do i 
    inflfn = inflfns[i]


    periods_metrics = mapreduce(merge, EVAL_PERIODS) do period
        # Ventana de evaluación
        eval_window = eval_periods(gtdata_eval, period)
        # Trayectorias combinadas
        metrics = eval_metrics(tray_infl[eval_window, i:i, :], tray_infl_pob[eval_window], prefix = period_tag(period))
    end

    metrics_df = DataFrame(periods_metrics)
    metrics_df[!, :measure] .= measure_name(inflfn)
    metrics_df
end;

In [84]:
select(indv_results, :measure, :gt_b00_mse, :gt_t0010_mse, :gt_b10_mse, :mse)

,measure,gt_b00_mse,gt_t0010_mse,gt_b10_mse
,String,Float32,Float32,Float32
1,Percentil equiponderado 72.4,0.243393,1.1195,0.0843358
2,Percentil ponderado 70.0,0.474509,1.3196,0.29442
3,"Media Truncada Equiponderada (58.76, 83.15)",0.224129,1.13039,0.0715498
4,"Media Truncada Ponderada (21.0, 95.89)",0.383119,0.872381,0.14664
5,"Inflación de exclusión dinámica (0.32, 1.68)",0.36609,1.14291,0.123634
6,"Exclusión fija de gastos básicos IPC (14, 14)",0.772971,0.982231,0.392376
7,MAI óptima MSE 2018,0.244553,0.76268,0.125143


In [85]:
select(indv_results, :measure, :mse)

,measure,mse
,String,Float32
1,Percentil equiponderado 72.4,0.209768
2,Percentil ponderado 70.0,0.429384
3,"Media Truncada Equiponderada (58.76, 83.15)",0.195036
4,"Media Truncada Ponderada (21.0, 95.89)",0.294061
5,"Inflación de exclusión dinámica (0.32, 1.68)",0.288
6,"Exclusión fija de gastos básicos IPC (14, 14)",0.601866
7,MAI óptima MSE 2018,0.212604


### Métricas de la combinación óptima MSE

In [86]:
optmse2022 = wload(datadir(config_savepath, "optmse2022", "optmse2022.jld2"), "optmse2022")
components_df = components(optmse2022)

,measure,weights
,String,Float32
1,Percentil equiponderado 72.4,4.07182e-6
2,Percentil ponderado 70.0,1.64225e-6
3,"Media Truncada Equiponderada (58.76, 83.15)",0.727412
4,"Media Truncada Ponderada (21.0, 95.89)",4.53039e-6
5,"Inflación de exclusión dinámica (0.32, 1.68)",0.0160966
6,MAI óptima MSE 2018,0.256481
7,"Exclusión fija de gastos básicos IPC (14, 14)",0.0


In [87]:
combine_df = leftjoin(components_df, indv_results, on=:measure)
select(combine_df, :measure, :weights)

,measure,weights
,String,Float32
1,Percentil equiponderado 72.4,4.07182e-6
2,Percentil ponderado 70.0,1.64225e-6
3,"Media Truncada Equiponderada (58.76, 83.15)",0.727412
4,"Media Truncada Ponderada (21.0, 95.89)",4.53039e-6
5,"Inflación de exclusión dinámica (0.32, 1.68)",0.0160966
6,"Exclusión fija de gastos básicos IPC (14, 14)",0.0
7,MAI óptima MSE 2018,0.256481


In [88]:
# Revisar que el orden en el DataFrame de combinación sea el mismo que en el
# vector de funciones de inflación inflfns, ya que es el orden en que se
# encuentran las trayectorias de simulación en el arreglo tray_infl
@assert combine_df.measure == measure_name.(inflfns) "Error en orden"

In [89]:
# Pesos de la combinación óptima
mse_weights = combine_df.weights

EVAL_PERIODS = [GT_EVAL_B00, GT_EVAL_T0010, GT_EVAL_B10, CompletePeriod()]

comb_metrics = mapreduce(merge, EVAL_PERIODS) do period
    # Ventana de evaluación
    eval_window = eval_periods(gtdata_eval, period)
    # Trayectorias combinadas
    tray_infl_opt = sum(tray_infl[eval_window, :, :] .* mse_weights', dims=2)
    metrics = eval_metrics(tray_infl_opt, tray_infl_pob[eval_window], prefix = period_tag(period))
end;

# Obtener un DataFrame de métricas de la combinación lineal CORR
comb_metrics_df = DataFrame(comb_metrics)
comb_metrics_df[!, :measure] .= measure_name(optmse2022)
select(comb_metrics_df, :measure, r"mse$")

,measure,gt_b00_mse,gt_b00_rmse,gt_b10_mse,gt_b10_rmse,gt_t0010_mse
,String,Float32,Float32,Float32,Float32,Float32
1,Subyacente óptima MSE 2022,0.221864,0.46519,0.0635692,0.249898,1.02145


In [90]:
# Combinar los resultados individuales con los de la combinación MSE
historic_df = [
    select(indv_results, :measure, :gt_b00_mse, :gt_t0010_mse, :gt_b10_mse, :mse); 
    select(comb_metrics_df, :measure, :gt_b00_mse, :gt_t0010_mse, :gt_b10_mse, :mse)
]

,measure,gt_b00_mse,gt_t0010_mse,gt_b10_mse
,String,Float32,Float32,Float32
1,Percentil equiponderado 72.4,0.243393,1.1195,0.0843358
2,Percentil ponderado 70.0,0.474509,1.3196,0.29442
3,"Media Truncada Equiponderada (58.76, 83.15)",0.224129,1.13039,0.0715498
4,"Media Truncada Ponderada (21.0, 95.89)",0.383119,0.872381,0.14664
5,"Inflación de exclusión dinámica (0.32, 1.68)",0.36609,1.14291,0.123634
6,"Exclusión fija de gastos básicos IPC (14, 14)",0.772971,0.982231,0.392376
7,MAI óptima MSE 2018,0.244553,0.76268,0.125143
8,Subyacente óptima MSE 2022,0.221864,1.02145,0.0635692


## Métricas en período de optimización (Dic-2001 - Dic-2018)

In [91]:
# Trayectorias de inflación y parámetrica
tray_infl = testdata["infl_18"]
tray_infl_pob = testdata["param_18"];

gtdata_eval = gtdata[Date(2018, 12)]
opt_period = EvalPeriod(Date(2001, 12), Date(2018, 12), "opt18")

opt18:Dec-01-Dec-18

### Métricas individuales del período de optimización

In [92]:
inflfns = [testconfig.inflfn.functions...]
EVAL_PERIODS = [opt_period]

indv_results = mapreduce(vcat, 1:length(inflfns)) do i 
    inflfn = inflfns[i]

    periods_metrics = mapreduce(merge, EVAL_PERIODS) do period
        # Ventana de evaluación
        eval_window = eval_periods(gtdata_eval, period)
        # Trayectorias combinadas
        metrics = eval_metrics(tray_infl[eval_window, i:i, :], tray_infl_pob[eval_window], prefix = period_tag(period))
    end

    metrics_df = DataFrame(periods_metrics)
    metrics_df[!, :measure] .= measure_name(inflfn)
    metrics_df
end;

In [93]:
select(indv_results, :measure, :opt18_mse)

,measure,opt18_mse
,String,Float32
1,Percentil equiponderado 72.4,0.24027
2,Percentil ponderado 70.0,0.410102
3,"Media Truncada Equiponderada (58.76, 83.15)",0.220518
4,"Media Truncada Ponderada (21.0, 95.89)",0.305004
5,"Inflación de exclusión dinámica (0.32, 1.68)",0.302949
6,"Exclusión fija de gastos básicos IPC (14, 14)",0.613884
7,MAI óptima MSE 2018,0.209567


### Métricas de la combinación óptima MSE en período de optimización

In [94]:
combine_df = leftjoin(components_df, indv_results, on=:measure)
select(combine_df, :measure, :weights)

,measure,weights
,String,Float32
1,Percentil equiponderado 72.4,4.07182e-6
2,Percentil ponderado 70.0,1.64225e-6
3,"Media Truncada Equiponderada (58.76, 83.15)",0.727412
4,"Media Truncada Ponderada (21.0, 95.89)",4.53039e-6
5,"Inflación de exclusión dinámica (0.32, 1.68)",0.0160966
6,"Exclusión fija de gastos básicos IPC (14, 14)",0.0
7,MAI óptima MSE 2018,0.256481


In [95]:
# Revisar que el orden en el DataFrame de combinación sea el mismo que en el
# vector de funciones de inflación inflfns, ya que es el orden en que se
# encuentran las trayectorias de simulación en el arreglo tray_infl
@assert combine_df.measure == measure_name.(inflfns) "Error en orden"

In [96]:
# Pesos de la combinación óptima
mse_weights = combine_df.weights

EVAL_PERIODS = [opt_period]

comb_metrics = mapreduce(merge, EVAL_PERIODS) do period
    # Ventana de evaluación
    eval_window = eval_periods(gtdata_eval, period)
    # Trayectorias combinadas
    tray_infl_opt = sum(tray_infl[eval_window, :, :] .* mse_weights', dims=2)
    metrics = eval_metrics(tray_infl_opt, tray_infl_pob[eval_window], prefix = period_tag(period))
end;

# Obtener un DataFrame de métricas de la combinación lineal CORR
comb_metrics_df = DataFrame(comb_metrics)
comb_metrics_df[!, :measure] .= measure_name(optmse2022)
select(comb_metrics_df, :measure, :opt18_mse)

,measure,opt18_mse
,String,Float32
1,Subyacente óptima MSE 2022,0.203698


In [97]:
# Combinar los resultados individuales con los de la combinación MSE
optim_period_df = [
    select(indv_results, :measure, :opt18_mse); 
    select(comb_metrics_df, :measure, :opt18_mse)
]

,measure,opt18_mse
,String,Float32
1,Percentil equiponderado 72.4,0.24027
2,Percentil ponderado 70.0,0.410102
3,"Media Truncada Equiponderada (58.76, 83.15)",0.220518
4,"Media Truncada Ponderada (21.0, 95.89)",0.305004
5,"Inflación de exclusión dinámica (0.32, 1.68)",0.302949
6,"Exclusión fija de gastos básicos IPC (14, 14)",0.613884
7,MAI óptima MSE 2018,0.209567
8,Subyacente óptima MSE 2022,0.203698


## Combinar las métricas en un DataFrame de salida

In [99]:
final_df = leftjoin(historic_df, optim_period_df, on=:measure)

,measure,gt_b00_mse,gt_t0010_mse,gt_b10_mse
,String,Float32,Float32,Float32
1,Percentil equiponderado 72.4,0.243393,1.1195,0.0843358
2,Percentil ponderado 70.0,0.474509,1.3196,0.29442
3,"Media Truncada Equiponderada (58.76, 83.15)",0.224129,1.13039,0.0715498
4,"Media Truncada Ponderada (21.0, 95.89)",0.383119,0.872381,0.14664
5,"Inflación de exclusión dinámica (0.32, 1.68)",0.36609,1.14291,0.123634
6,"Exclusión fija de gastos básicos IPC (14, 14)",0.772971,0.982231,0.392376
7,MAI óptima MSE 2018,0.244553,0.76268,0.125143
8,Subyacente óptima MSE 2022,0.221864,1.02145,0.0635692


In [100]:
# Guardar los resultados en un archivo CSV
function save_csv(file::AbstractString, df::DataFrame)
    encoding = enc"ISO-8859-1"
    @info "Saving file with $encoding" file
    open(file, encoding, "w") do io 
        CSV.write(io, df)
    end
end

metrics_savepath = mkpath(datadir("updates", "metrics"))
save_csv(
    joinpath(metrics_savepath, "optmse2022_metrics.csv"),
    final_df
)

┌ Info: Saving file with ISO-8859-1
│   file = c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\metrics\optmse2022_metrics.csv
└ @ Main c:\Users\RRCP\Documents\HEMI\HEMI\notebooks\historic-combination-metrics\mse-combination-metrics.ipynb:4


StringEncoder{UTF-8, ISO-8859-1}(IOStream(<file c:\Users\RRCP\Documents\HEMI\HEMI\data\updates\metrics\optmse2022_metrics.csv>))